
# <p style="text-align: center;">Large Weather and Decrees</p>

to do : enrichir avec latitude longitude, prendre directement les missing en dataframe et les rajouter à la mano, laisser en commentaire le truc qui les enrichie

In [1]:
#!pip install cartopy
#!pip install imageio

## Import libraries

In [2]:
import pandas as pd
import numpy as np

import glob
import os

import requests

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import zipfile

import xarray as xr

import folium
from folium.plugins import MarkerCluster

import pandas as pd
from geopy.geocoders import Nominatim

import geopandas as gpd
import cartopy.crs as ccrs

import imageio

from sklearn.neighbors import KDTree
from sklearn.neighbors import BallTree

from sklearn.preprocessing import RobustScaler

## Set parameters

In [6]:
decree_filename_base = 'arrete_'
decrees_folder_name = './../../data/raw/decrees'
communes_folder_name = './../../data/raw/opendatasoft'
weather_folder_name = './../../data/raw/weather/era5'
weather_ncfiles_folder_name = './../../data/raw/weather/era5/ncfiles2'
processed_data_folder_name = './../../data/processed'
output_data_folder_name = './../../data/processed/output'
decrees_filename = 'decrees.parquet'
decrees_locations_filename = 'decrees_locations.parquet'
communes_csv_filename = 'correspondance-code-insee-code-postal-202410.csv'
weather_filename = 'weather2.parquet'
weather_shema_filename = 'weather_shema2.csv'
weather_yearly_filename = 'weather2_yearly.parquet'
weather_yearly_shema_filename = 'weather_shema2_yearly.csv'
drought_filename = 'drought.parquet'
drought_shema_filename = 'drought_shema.csv'
drought_yearly_filename = 'drought_yearly.parquet'
drought_yearly_shema_filename = 'drought_yearly_shema.csv'
#weather_zip_file = "193fcd51a8958175843ecbbcaba057c8.zip"

## Load Weather data

| Column    | Description                                      |
|-----------|--------------------------------------------------|
| date      | The date of the observation.                     |
| latitude  | The latitude coordinate of the observation point.|
| longitude | The longitude coordinate of the observation point.|
| number    | A unique identifier for the observation.         |
| expver    | Experiment version number.                       |
| u10       | 10-meter U-component of wind (eastward wind).    |
| v10       | 10-meter V-component of wind (northward wind).   |
| t2m       | 2-meter temperature (air temperature at 2 meters above the surface).|
| sp        | Surface pressure.                                |
| tp        | Total precipitation.                             |
| e         | Evaporation.                                     |
| sro       | Surface runoff.                                  |
| tcrw      | Total column water vapor.                        |
| stl1      | Soil temperature level 1.                        |
| stl2      | Soil temperature level 2.                        |
| slt       | Soil type.                                       |
| swvl1     | Volumetric soil water layer 1.                   |
| swvl2     | Volumetric soil water layer 2.                   |
| cvh       | High vegetation cover.                           |
| cvl       | Low vegetation cover.                            |
| tvh       | High vegetation type.                            |
| tvl       | Low vegetation type.                             |


In [7]:
df_yearly_weather = pd.read_parquet(os.path.join(processed_data_folder_name, weather_yearly_filename))
weather_yearly_shema = pd.read_csv(os.path.join(processed_data_folder_name, weather_yearly_shema_filename), index_col=0).squeeze("columns")
df_yearly_weather = df_yearly_weather.astype(weather_yearly_shema.to_dict())

In [8]:
df_yearly_weather.head()

,latitude,longitude,year,t2m_z_sum,t2m_z_mean,tp_z_sum,tp_z_mean,e_z_sum,e_z_mean,pev_z_sum,...,stl1_sum,stl1_mean,stl2_sum,stl2_mean,slt_sum,slt_mean,swvl1_sum,swvl1_mean,swvl2_sum,swvl2_mean
0,42.5,-5.0,1940,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,...,131.224976,10.935414,131.221069,10.935089,36.0,3.0,3.491412,0.290951,3.582633,0.298553
1,42.5,-5.0,1941,0.000000,0.000000,-1.414214,-0.117851,1.414214,0.117851,-1.414214,...,130.369019,10.864085,129.770386,10.814198,36.0,3.0,3.291583,0.274299,3.486492,0.290541
2,42.5,-5.0,1942,2.590982,0.215915,0.948012,0.079001,-2.155737,-0.179645,-0.856798,...,132.005249,11.000438,131.522339,10.960195,36.0,3.0,3.473109,0.289426,3.584625,0.298719
3,42.5,-5.0,1943,3.948252,0.329021,-5.062950,-0.421913,-0.876471,-0.073039,-7.460276,...,143.968140,11.997345,143.213501,11.934459,36.0,3.0,3.147441,0.262287,3.267897,0.272325
4,42.5,-5.0,1944,-0.020748,-0.001729,-2.657244,-0.221437,5.106711,0.425559,-5.488278,...,133.577759,11.131480,133.382690,11.115224,36.0,3.0,2.850830,0.237569,2.922482,0.243540


In [9]:
df_yearly_weather.columns

Index(['latitude', 'longitude', 'year', 't2m_z_sum', 't2m_z_mean', 'tp_z_sum',
       'tp_z_mean', 'e_z_sum', 'e_z_mean', 'pev_z_sum', 'pev_z_mean',
       'stl1_z_sum', 'stl1_z_mean', 'stl2_z_sum', 'stl2_z_mean', 'slt_z_sum',
       'slt_z_mean', 'swvl1_z_sum', 'swvl1_z_mean', 'swvl2_z_sum',
       'swvl2_z_mean', 't2m_sum', 't2m_mean', 'tp_sum', 'tp_mean', 'e_sum',
       'e_mean', 'pev_sum', 'pev_mean', 'stl1_sum', 'stl1_mean', 'stl2_sum',
       'stl2_mean', 'slt_sum', 'slt_mean', 'swvl1_sum', 'swvl1_mean',
       'swvl2_sum', 'swvl2_mean'],
      dtype='object')

In [10]:
df_yearly_weather.dtypes

latitude        float64
longitude       float64
year              int32
t2m_z_sum       float64
t2m_z_mean      float64
tp_z_sum        float64
tp_z_mean       float64
e_z_sum         float64
e_z_mean        float64
pev_z_sum       float64
pev_z_mean      float64
stl1_z_sum      float64
stl1_z_mean     float64
stl2_z_sum      float64
stl2_z_mean     float64
slt_z_sum       float64
slt_z_mean      float64
swvl1_z_sum     float64
swvl1_z_mean    float64
swvl2_z_sum     float64
swvl2_z_mean    float64
t2m_sum         float32
t2m_mean        float32
tp_sum          float32
tp_mean         float32
e_sum           float32
e_mean          float32
pev_sum         float32
pev_mean        float32
stl1_sum        float32
stl1_mean       float32
stl2_sum        float32
stl2_mean       float32
slt_sum         float32
slt_mean        float32
swvl1_sum       float32
swvl1_mean      float32
swvl2_sum       float32
swvl2_mean      float32
dtype: object

In [11]:
print(df_yearly_weather.isna().sum())  # Count missing values

latitude           0
longitude          0
year               0
t2m_z_sum          0
t2m_z_mean      1925
tp_z_sum           0
tp_z_mean       1925
e_z_sum            0
e_z_mean        1925
pev_z_sum          0
pev_z_mean      1925
stl1_z_sum         0
stl1_z_mean     1925
stl2_z_sum         0
stl2_z_mean     1925
slt_z_sum          0
slt_z_mean      1925
swvl1_z_sum        0
swvl1_z_mean    1925
swvl2_z_sum        0
swvl2_z_mean    1925
t2m_sum            0
t2m_mean           0
tp_sum             0
tp_mean            0
e_sum              0
e_mean             0
pev_sum            0
pev_mean           0
stl1_sum           0
stl1_mean          0
stl2_sum           0
stl2_mean          0
slt_sum            0
slt_mean           0
swvl1_sum          0
swvl1_mean         0
swvl2_sum          0
swvl2_mean         0
dtype: int64


In [12]:
# Filter the DataFrame to exclude the year 1940
df_yearly_weather[df_yearly_weather['year'] != 1940].isna().sum().sum()




0

In [13]:
df_yearly_weather.drop(df_yearly_weather[df_yearly_weather['year'] == 1940].index, inplace=True)

In [14]:
df_yearly_weather.isna().sum().sum()

0

### Note

>There are no missing values in weather data because they come from monthly ERA5 simulations based on real data. only missing in 1940 du to rolling window for featured parameters.

## Import Decrees

In [15]:


df_drought_yearly = pd.read_parquet(os.path.join(processed_data_folder_name, drought_yearly_filename))
drought_schema_yearly = pd.read_csv(os.path.join(processed_data_folder_name, drought_yearly_shema_filename), index_col=0).squeeze("columns")
df_drought_yearly = df_drought_yearly.astype(drought_schema_yearly.to_dict())

In [17]:
df_drought_yearly.head()

,insee,nom_commune,debut_evenement,fin_evenement,date_arrete,date_parution_jo,nom_peril,code_peril,franchise,code_nor,decision,event_length_days,year,year_count,year_duration
43673,12018,BALAGUIER D OLT,1990-01-01,1990-09-30,2001-11-15,2001-12-01,Sécheresse,SEC,Simple,INTE0100649A,Reconnue,272,1990,1,1
43674,12018,BALAGUIER D OLT,1992-03-01,1992-06-30,2001-11-15,2001-12-01,Sécheresse,SEC,Simple,INTE0100649A,Reconnue,121,1992,1,1
43677,17118,CORIGNAC,1989-05-01,1989-12-31,2001-11-15,2001-12-01,Sécheresse,SEC,Simple,INTE0100649A,Reconnue,244,1989,1,1
43678,17203,LANDRAIS,1990-01-01,1990-09-30,2001-11-15,2001-12-01,Sécheresse,SEC,Simple,INTE0100649A,Reconnue,272,1990,1,1
43679,17260,NEUVICQ,1989-05-01,1989-12-31,2001-11-15,2001-12-01,Sécheresse,SEC,Simple,INTE0100649A,Reconnue,244,1989,1,1


| Column    | Description                                      |
|-----------|--------------------------------------------------|
| date      | The date of the observation.                     |
| latitude  | The latitude coordinate of the observation point.|
| longitude | The longitude coordinate of the observation point.|
| number    | A unique identifier for the observation.         |
| expver    | Experiment version number.                       |
| u10       | 10-meter U-component of wind (eastward wind).    |
| v10       | 10-meter V-component of wind (northward wind).   |
| t2m       | 2-meter temperature (air temperature at 2 meters above the surface).|
| sp        | Surface pressure.                                |
| tp        | Total precipitation.                             |
| e         | Evaporation.                                     |
| sro       | Surface runoff.                                  |
| tcrw      | Total column water vapor.                        |
| stl1      | Soil temperature level 1.                        |
| stl2      | Soil temperature level 2.                        |
| slt       | Soil type.                                       |
| swvl1     | Volumetric soil water layer 1.                   |
| swvl2     | Volumetric soil water layer 2.                   |
| cvh       | High vegetation cover.                           |
| cvl       | Low vegetation cover.                            |
| tvh       | High vegetation type.                            |
| tvl       | Low vegetation type.                             |


## Import Locations

In [18]:
df_communes_location = pd.read_csv(os.path.join(communes_folder_name, communes_csv_filename), sep=";")

In [19]:
df_communes_location.head()

,Code INSEE,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,ID Geofla,Code Commune,Code Canton,Code Arrondissement,Code Département,Code Région
0,64012,64130,AINHARP,['PYRENEES-ATLANTIQUES'],['AQUITAINE'],['Commune simple'],232.0,1431.0,0.1,"43.25473886787848, -0.935719451957656","{""coordinates"": [[[-0.923667598351223, 43.2284...",3178,12,20,2,64,72
1,2A108,20126,EVISA,['CORSE-DU-SUD'],['CORSE'],['Commune simple'],1160.0,6720.0,0.2,"42.28525022904632, 8.820396904179216","{""coordinates"": [[[8.813203180956721, 42.23916...",1332,108,35,1,2A,94
2,28232,28240,MANOU,['EURE-ET-LOIR'],['CENTRE'],['Commune simple'],237.0,1361.0,0.6,"48.53638563478625, 0.962933887077251","{""coordinates"": [[[0.9962700204594641, 48.5274...",4524,232,17,4,28,24
3,62761,62128,SAINT-MARTIN-SUR-COJEUL,['PAS-DE-CALAIS'],['NORD-PAS-DE-CALAIS'],['Commune simple'],86.0,343.0,0.2,"50.22864782200462, 2.851919793535168","{""coordinates"": [[[2.862290324179924, 50.21323...",18319,761,20,1,62,31
4,56219,56460,SAINT-GUYOMARD,['MORBIHAN'],['BRETAGNE'],['Commune simple'],79.0,1979.0,1.0,"47.77934192534663, -2.502103749642248","{""coordinates"": [[[-2.481587975007875, 47.7614...",4824,219,19,3,56,53


## Decrees  Locations

In [35]:
def check_insee_coverage(df1, df2, insee_col1, insee_col2):
    # Step 1: Create copies of the dataframes to avoid modifying the input dataframes
    df1_copy = df1.copy()
    df2_copy = df2.copy()

    # Step 2: Remove spaces before and after in the INSEE code columns and remove leading zeros to ensure proper comparison
    df1_copy[insee_col1] = df1_copy[insee_col1].str.strip().str.lstrip('0')
    df2_copy[insee_col2] = df2_copy[insee_col2].str.strip().str.lstrip('0')

    # Step 3: Get the list of INSEE codes from df2
    insee_codes_df2 = df2_copy[insee_col2].unique()

    # Step 4: Check if all INSEE codes in df1 are present in df2
    all_in_df2 = df1_copy[insee_col1].isin(insee_codes_df2)

    # Step 5: Return the communes in df1 that are not in df2
    missing_communes = df1_copy[~all_in_df2]
    return missing_communes

# Example usage of the function
missing_communes = check_insee_coverage(df_drought_yearly, df_communes_location, 'insee', 'Code INSEE')
print(missing_communes[['insee', 'nom_commune']])


        insee        nom_commune
69059   79065        CHAMBROUTET
69059   79065        CHAMBROUTET
69706   59248       FORT MARDYCK
69706   59248       FORT MARDYCK
75499   13055          MARSEILLE
75499   13055          MARSEILLE
75499   13055          MARSEILLE
75783   62575          MOLINGHEM
94181   59248       FORT MARDYCK
94181   59248       FORT MARDYCK
97902   59540  SAINT POL SUR MER
97902   59540  SAINT POL SUR MER
97902   59540  SAINT POL SUR MER
165856  13055          MARSEILLE
165856  13055          MARSEILLE
165857  13055          MARSEILLE
169786  59248       FORT MARDYCK
175422  13055          MARSEILLE
193312  13055          MARSEILLE
193313  13055          MARSEILLE
193314  13055          MARSEILLE
194390  13055          MARSEILLE
194391  13055          MARSEILLE
205082  13055          MARSEILLE
237044  13055          MARSEILLE
241976  13055          MARSEILLE
260470  13055          MARSEILLE
276234  13055          MARSEILLE
290237  13055          MARSEILLE


In [36]:
unique_values = missing_communes[['insee', 'nom_commune']].drop_duplicates()
print(unique_values)


       insee        nom_commune
69059  79065        CHAMBROUTET
69706  59248       FORT MARDYCK
75499  13055          MARSEILLE
75783  62575          MOLINGHEM
97902  59540  SAINT POL SUR MER


In [37]:
missing_communes

,insee,nom_commune,debut_evenement,fin_evenement,date_arrete,date_parution_jo,nom_peril,code_peril,franchise,code_nor,decision,event_length_days,year,year_count,year_duration
69059,79065,CHAMBROUTET,1989-05-01,1990-12-31,1991-12-04,1991-12-27,Sécheresse,SEC,Simple,INTX9110334A,Reconnue,609,1989,1,2
69059,79065,CHAMBROUTET,1989-05-01,1990-12-31,1991-12-04,1991-12-27,Sécheresse,SEC,Simple,INTX9110334A,Reconnue,609,1990,2,2
69706,59248,FORT MARDYCK,1990-01-01,1991-12-31,1992-07-31,1992-08-18,Sécheresse,SEC,Simple,INTX9210277A,Reconnue,729,1990,1,2
69706,59248,FORT MARDYCK,1990-01-01,1991-12-31,1992-07-31,1992-08-18,Sécheresse,SEC,Simple,INTX9210277A,Reconnue,729,1991,2,2
75499,13055,MARSEILLE,1989-05-01,1991-12-31,1993-01-25,1993-02-07,Sécheresse,SEC,Simple,INTE9300001A,Reconnue,974,1989,1,3
75499,13055,MARSEILLE,1989-05-01,1991-12-31,1993-01-25,1993-02-07,Sécheresse,SEC,Simple,INTE9300001A,Reconnue,974,1990,2,3
75499,13055,MARSEILLE,1989-05-01,1991-12-31,1993-01-25,1993-02-07,Sécheresse,SEC,Simple,INTE9300001A,Reconnue,974,1991,3,3
75783,62575,MOLINGHEM,1991-01-01,1991-12-31,1993-01-25,1993-02-07,Sécheresse,SEC,Simple,INTE9300001A,Reconnue,364,1991,1,1
94181,59248,FORT MARDYCK,1992-01-01,1993-12-31,1996-02-02,1996-02-14,Sécheresse,SEC,Simple,INTE9600039A,Reconnue,730,1992,1,2
94181,59248,FORT MARDYCK,1992-01-01,1993-12-31,1996-02-02,1996-02-14,Sécheresse,SEC,Simple,INTE9600039A,Reconnue,730,1993,2,2


In [24]:
df_drought_yearly['insee'].dtypes

dtype('O')

In [25]:
df_communes_location['Code INSEE'].dtypes

dtype('O')

In [33]:
df_communes_location[df_communes_location['Code INSEE'].str.strip()=='1339']

,Code INSEE,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,ID Geofla,Code Commune,Code Canton,Code Arrondissement,Code Département,Code Région


In [34]:
print(df_communes_location.iloc[18260:18270])


      Code INSEE Code Postal                    Commune  \
18260      76612       76340       SAINT-MARTIN-AU-BOSC   
18261      76186       76390                 CONTEVILLE   
18262      97353       97370                MARIPASOULA   
18263      01339       01600              SAINT-BERNARD   
18264      74190       74440                   MORILLON   
18265      67142       67480                 FORT-LOUIS   
18266      21556       21270          SAINT-LEGER-TRIEY   
18267      62139       62575                BLENDECQUES   
18268      64189       64500                    CIBOURE   
18269      47256       47210  SAINT-MARTIN-DE-VILLEREAL   

                    Département                  Région                Statut  \
18260        ['SEINE-MARITIME']     ['HAUTE-NORMANDIE']    ['Commune simple']   
18261        ['SEINE-MARITIME']     ['HAUTE-NORMANDIE']    ['Commune simple']   
18262                ['GUYANE']              ['GUYANE']  ['Chef-lieu canton']   
18263                   ['

In [38]:
import pandas as pd

data = {
    'insee': [79065, 59248, 13055, 62575, 59540],
    'nom_commune': ['Chambrouet', 'Fort-Mardyck', 'Marseille', 'Molinghem', 'Saint-Pol-sur-Mer'],
    'latitude': [48.85341, 50.5432, 43.2965, 50.6542, 50.9912],
    'longitude': [2.3488, 2.9889, 5.3698, 2.2298, 2.3432],
    'department': ['Charente-Maritime', 'Nord', 'Bouches-du-Rhône', 'Pas-de-Calais', 'Nord'],
    'region': ['Nouvelle-Aquitaine', 'Hauts-de-France', 'Provence-Alpes-Côte d\'Azur', 'Hauts-de-France', 'Hauts-de-France']
}

df = pd.DataFrame(data)
print(df)


   insee        nom_commune  latitude  longitude         department  \
0  79065         Chambrouet  48.85341     2.3488  Charente-Maritime   
1  59248       Fort-Mardyck  50.54320     2.9889               Nord   
2  13055          Marseille  43.29650     5.3698   Bouches-du-Rhône   
3  62575          Molinghem  50.65420     2.2298      Pas-de-Calais   
4  59540  Saint-Pol-sur-Mer  50.99120     2.3432               Nord   

                       region  
0          Nouvelle-Aquitaine  
1             Hauts-de-France  
2  Provence-Alpes-Côte d'Azur  
3             Hauts-de-France  
4             Hauts-de-France  


In [39]:
import folium

# Initialize the map centered around France
m = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Data for communes
data = {
    'insee': [79065, 59248, 13055, 62575, 59540],
    'nom_commune': ['Chambrouet', 'Fort-Mardyck', 'Marseille', 'Molinghem', 'Saint-Pol-sur-Mer'],
    'latitude': [48.85341, 50.5432, 43.2965, 50.6542, 50.9912],
    'longitude': [2.3488, 2.9889, 5.3698, 2.2298, 2.3432],
    'department': ['Charente-Maritime', 'Nord', 'Bouches-du-Rhône', 'Pas-de-Calais', 'Nord'],
    'region': ['Nouvelle-Aquitaine', 'Hauts-de-France', 'Provence-Alpes-Côte d\'Azur', 'Hauts-de-France', 'Hauts-de-France'],
    'department_number': ['17', '59', '13', '62', '59']
}

# Add markers for each commune with department and region info
for i in range(len(data['nom_commune'])):
    folium.Marker(
        location=[data['latitude'][i], data['longitude'][i]],
        popup=f"{data['nom_commune'][i]}, Dept: {data['department'][i]} ({data['department_number'][i]}), Region: {data['region'][i]}"
    ).add_to(m)

# Save the map to an HTML file
m.save('communes_map.html')


In [40]:
m

In [42]:
from geopy.geocoders import Nominatim
import pandas as pd
import folium

# Create a DataFrame with the INSEE codes and commune names
data = {
    'insee': ['79065', '59248', '13055', '62575', '59540'],
    'nom_commune': ['CHAMBROUTET', 'FORT MARDYCK', 'MARSEILLE', 'MOLINGHEM', 'SAINT POL SUR MER']
}
df_communes = pd.DataFrame(data)

# Initialize Nominatim API
geolocator = Nominatim(user_agent="insee_locator")

# Function to get latitude and longitude
def get_lat_long(commune_name):
    try:
        location = geolocator.geocode(f"{commune_name}, France")
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        return None, None

# Function to get department and region information
def get_department_region(commune_name):
    try:
        location = geolocator.geocode(f"{commune_name}, France")
        if location:
            location_detail = geolocator.reverse((location.latitude, location.longitude), language='fr')
            if location_detail and 'address' in location_detail.raw:
                address = location_detail.raw['address']
                departement = address.get('county', None)
                departement_number = address.get('state_code', None)
                region = address.get('state', None)
                return departement, departement_number, region
        return None, None, None
    except Exception as e:
        return None, None, None

# Apply the function to get latitude, longitude, department, and region
df_communes[['latitude', 'longitude']] = df_communes['nom_commune'].apply(
    lambda x: pd.Series(get_lat_long(x))
)

df_communes[['departement', 'departement_number', 'region']] = df_communes['nom_commune'].apply(
    lambda x: pd.Series(get_department_region(x))
)

# Create a Folium map centered at an approximate central location in France
map_center = [46.603354, 1.888334]
communes_map = folium.Map(location=map_center, zoom_start=6)

# Add markers for each commune with a popup containing its information
for _, row in df_communes.iterrows():
    if pd.notna(row['latitude']) and pd.notna(row['longitude']):
        popup_text = (
            f"Commune: {row['nom_commune']}<br>"
            f"INSEE Code: {row['insee']}<br>"
            f"Department: {row['departement']} ({row['departement_number']})<br>"
            f"Region: {row['region']}"
        )
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=folium.Popup(popup_text, max_width=300)
        ).add_to(communes_map)

# Save the map to an HTML file
communes_map.save('communes_map.html')

# Print the DataFrame with all the information
print(df_communes)

# Example output (Note: The actual coordinates and department information depend on the geocoder's response)
#     insee          nom_commune   latitude   longitude      departement   departement_number   region
# 0  79065         CHAMBROUTET  46.7890     -0.5678      Deux-Sèvres       79                Nouvelle-Aquitaine
# 1  59248        FORT MARDYCK  51.0356      2.3314      Nord             59                Hauts-de-France
# 2  13055           MARSEILLE  43.2965      5.3698      Bouches-du-Rhône 13                Provence-Alpes-Côte d'Azur
# 3  62575           MOLINGHEM  50.6631      2.3387      Pas-de-Calais    62                Hauts-de-France
# 4  59540   SAINT POL SUR MER  51.0250      2.3333      Nord             59                Hauts-de-France


   insee        nom_commune   latitude  longitude       departement  \
0  79065        CHAMBROUTET  46.886384  -0.468974       Deux-Sèvres   
1  59248       FORT MARDYCK  51.029283   2.306730              Nord   
2  13055          MARSEILLE  43.296174   5.369953  Bouches-du-Rhône   
3  62575          MOLINGHEM  50.610908   2.448105     Pas-de-Calais   
4  59540  SAINT POL SUR MER  51.027373   2.342220              Nord   

  departement_number                      region  
0               None          Nouvelle-Aquitaine  
1               None             Hauts-de-France  
2               None  Provence-Alpes-Côte d'Azur  
3               None             Hauts-de-France  
4               None             Hauts-de-France  


In [43]:
communes_map

In [ ]:
import pandas as pd

data = {
    'insee': [79065, 59248, 13055, 62575, 59540],
    'nom_commune': ['CHAMBROUTET', 'FORT MARDYCK', 'MARSEILLE', 'MOLINGHEM', 'SAINT POL SUR MER'],
    'latitude': [46.7890, 51.0356, 43.2965, 50.6631, 51.0250],
    'longitude': [-0.5678, 2.3314, 5.3698, 2.3387, 2.3333],
    'departement': ['Deux-Sèvres', 'Nord', 'Bouches-du-Rhône', 'Pas-de-Calais', 'Nord'],
    'departement_number': [79, 59, 13, 62, 59],
    'region': ['Nouvelle-Aquitaine', 'Hauts-de-France', 'Provence-Alpes-Côte d\'Azur', 'Hauts-de-France', 'Hauts-de-France']
}

df = pd.DataFrame(data)
print(df)
